## **The block below shows the DeepSurv_logger**

In [0]:
import logging
import tensorboard_logger 
from collections import defaultdict
import sys
import math

class DeepSurvLogger():
    def __init__(self, name):
        self.logger         = logging.getLogger(name)
        self.history = {}

    def logMessage(self,message):
        self.logger.info(message)

    def print_progress_bar(self, step, max_steps, loss = None, ci = None, bar_length = 25, char = '*'):
        progress_length = int(bar_length * step / max_steps)
        progress_bar = [char] * (progress_length) + [' '] * (bar_length - progress_length)
        space_padding = int(math.log10(max_steps))
        if step > 0:
            space_padding -= int(math.log10(step))
        space_padding = ''.join([' '] * space_padding)
        message = "Training step %d/%d %s|" % (step, max_steps, space_padding) + ''.join(progress_bar) + "|"
        if loss:
            message += " - loss: %.4f" % loss
        if ci:
            message += " - ci: %.4f" % ci

        self.logger.info(message)

    def logValue(self, key, value, step):
        pass

    def shutdown(self):
        logging.shutdown()

class TensorboardLogger(DeepSurvLogger):
    def __init__(self, name, logdir, max_steps = None, update_freq = 10):
        self.max_steps = max_steps

        self.logger         = logging.getLogger(name)
        self.logger.setLevel(logging.DEBUG)
        ch = logging.StreamHandler(sys.stdout)
        format = logging.Formatter("%(asctime)s - %(message)s")
        ch.setFormatter(format)
        self.logger.addHandler(ch)

        self.update_freq    = update_freq

        self.tb_logger = tensorboard_logger.Logger(logdir)

        self.history = defaultdict(list)

    def logValue(self, key, value, step):
        self.tb_logger.log_value(key, value, step)
        self.history[key].append((step, value))

In [0]:
!pip install tensorboard_logger 

## **Utility functions for running DeepSurv experiments**

In [0]:
!pip install lasagne

     |████████████████████████████████| 133kB 2.7MB/s 
  Created wheel for lasagne: filename=Lasagne-0.1-cp36-none-any.whl size=79275 sha256=154ff8bf9cd00a171130835224ea3acbb8256e3b66cd18ecd1ff0f2fa40cce49
  Stored in directory: /root/.cache/pip/wheels/a5/8e/31/b4cae7e5507f8582e77d7f5cf2815be8820ccacfa0519ca60c
Successfully built lasagne


In [0]:
!pip install downsample

ERROR: Could not find a version that satisfies the requirement downsample (from versions: none)
ERROR: No matching distribution found for downsample


In [0]:
pip install --user downsample

ERROR: Could not find a version that satisfies the requirement downsample (from versions: none)
ERROR: No matching distribution found for downsample


# Utility functions for visualizing results of DeepSurv experiments

In [0]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pylab

import numpy as np

import os

from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

def extract_value_list(arr):
    return list(np.array(arr)[:,1])

def plot_log(log):
    """
    Plots the training and validation curves for a network's loss function
    and calculated concordance index.

    Parameters:
        log: a dictionary with a list of values for any of the following keys:
            'train': training loss
            'valid': validation loss
            'train_ci': training concordance index
            VALID_CI: validation concordance index
    """
    TRAIN_LOSS = 'loss'
    TRAIN_CI = 'c-index'
    VALID_LOSS = 'valid_loss'
    VALID_CI = 'valid_c-index'

    num_epochs = len(log[TRAIN_LOSS])

    # Plots Negative Log Likelihood vs. Epoch
    fig, ax1 = plt.subplots()
    # plt.figure()
    handles = []
    if TRAIN_LOSS in log:
        epochs = range(num_epochs)
        values = extract_value_list(log[TRAIN_LOSS])
        train, = ax1.plot(epochs, values, 'b', label = 'Training')
        ax1.tick_params('y', colors='b')
        handles.append(train)
    if VALID_LOSS in log:
        ax2 = ax1.twinx()
        epochs = np.linspace(0,num_epochs-1,num=len(log[VALID_LOSS]))
        values = extract_value_list(log[VALID_LOSS])
        valid, = ax2.plot(epochs,values, 'r', label = 'Validation')
        ax2.tick_params('y', colors='r')
        handles.append(valid)
    plt.xlabel('Epoch')
    plt.ylabel('Negative Log Likelihood')
    plt.legend(handles=handles, loc = 0)

    # Plots Concordance Index vs. Epoch
    plt.figure()
    handles = []
    if TRAIN_CI in log:
        epochs = np.linspace(0,num_epochs-1,num=len(log[TRAIN_CI]))
        train, = plt.plot(epochs, extract_value_list(log[TRAIN_CI]), label = 'Training')
        handles.append(train)
    if VALID_CI in log:
        epochs = np.linspace(0,num_epochs-1,num=len(log[VALID_CI]))
        valid, = plt.plot(epochs, extract_value_list(log[VALID_CI]), label = 'Validation')
        handles.append(valid)
    plt.xlabel('Epoch')
    plt.ylabel('Concordance Index')
    plt.legend(handles = handles, loc = 4)

def plot_risk_model(x_0, x_1, hr, figsize=(4,3), clim = (-3,3), cmap = 'jet'):
    fig, ax = plt.subplots(figsize=figsize)
    plt.xlim(-1, 1)
    plt.xlabel('$x_0$', fontsize='large')
    plt.xticks(np.arange(-1, 1.5, .5))

    plt.ylim(-1, 1)
    plt.ylabel('$x_1$', fontsize='large')
    plt.yticks(np.arange(-1, 1.5, .5))
    
    im = plt.scatter(x=x_0, y=x_1, c=hr, marker='.', cmap=cmap)
    fig.colorbar(im)
    # plt.clim(0, 1)
    plt.clim(*clim)
    plt.tight_layout()
    return (fig, ax, im)

def save_fig(fig, fp):
    # TODO fit the pdf saving cutting off the x and y axis labels
    pp_true = PdfPages(fp)
    pp_true.savefig(fig, dpi=600)
    pp_true.close()

def plot_experiment_scatters(risk_fxn, dataset, norm_vals = None, output_file=None, 
    figsize = (4,3), clim=(-3,3), cmap = 'jet', plot_error=False, trt_idx = None):
    
    def norm_hr(hr):
        # return hr
        return hr - hr.mean();
        # return (hr - hr.min()) / (hr.max() - hr.min())

    x_0 = dataset['x'][:, 0]
    x_1 = dataset['x'][:, 1]

    # Plot model predictions
    x = dataset['x']
    if norm_vals:
        x = (x - norm_vals['mean']) / norm_vals['std']

    (head, tail) = os.path.split(output_file)

    if not trt_idx is None:
        trt_values = np.unique(x[:,trt_idx])
        for (idx,trt_value) in enumerate(trt_values):
            x_trt = np.copy(x)
            x_trt[:,trt_idx] = trt_value
            hr_trt = risk_fxn(x_trt)
            hr_trt = norm_hr(hr_trt)
            fig_trt, _, _ = plot_risk_model(x_0, x_1, hr_trt, figsize, clim, cmap)

            if output_file:
                save_fig(fig_trt, os.path.join(head, "treatment_%d_" % idx + tail))
    else:
        hr_pred = risk_fxn(x)
        hr_pred = norm_hr(hr_pred)
        fig_pred, _, _ = plot_risk_model(x_0, x_1, hr_pred, figsize, clim, cmap)

        if output_file:
            save_fig(fig_pred, os.path.join(head, "pred_" + tail))

    if 'hr' in dataset:
        hr_true = dataset['hr']
        hr_true = norm_hr(hr_true)
        fig_true, _, _ = plot_risk_model(x_0, x_1, hr_true, figsize, clim, cmap)

        if output_file:
            save_fig(fig_true, os.path.join(head, "true_" + tail))

        if plot_error:
            hr_error = np.abs(hr_true - hr_pred)
            fig_error, _, _ = plot_risk_model(x_0, x_1, hr_error, figsize, clim=(0,20), cmap = cmap)

            if output_file:
                save_fig(fig_error, os.path.join(head, "error_" + tail))

def plot_survival_curves(rec_t, rec_e, antirec_t, antirec_e, experiment_name = '', output_file = None):
    # Set-up plots
    plt.figure(figsize=(12,3))
    ax = plt.subplot(111)

    # Fit survival curves
    kmf = KaplanMeierFitter()
    kmf.fit(rec_t, event_observed=rec_e, label=' '.join([experiment_name, "Recommendation"]))   
    kmf.plot(ax=ax,linestyle="-")
    kmf.fit(antirec_t, event_observed=antirec_e, label=' '.join([experiment_name, "Anti-Recommendation"]))
    kmf.plot(ax=ax,linestyle="--")
    
    # Format graph
    plt.ylim(0,1);
    ax.set_xlabel('Timeline (months)',fontsize='large')
    ax.set_ylabel('Percentage of Population Alive',fontsize='large')
    
    # Calculate p-value
    results = logrank_test(rec_t, antirec_t, rec_e, antirec_e, alpha=.95)
    results.print_summary()

    # Location the label at the 1st out of 9 tick marks
    xloc = max(np.max(rec_t),np.max(antirec_t)) / 9
    if results.p_value < 1e-5:
        ax.text(xloc,.2,'$p < 1\mathrm{e}{-5}$',fontsize=20)
    else:
        ax.text(xloc,.2,'$p=%f$' % results.p_value,fontsize=20)
    plt.legend(loc='best',prop={'size':15})


    if output_file:
        plt.tight_layout()
        pylab.savefig(output_file)

In [0]:
!pip install lifelines

     |████████████████████████████████| 327kB 2.6MB/s 


### DataSet Exploration

In [0]:
from math import log, exp
import numpy as np

class SimulatedData:
    def __init__(self, hr_ratio,
        average_death = 5,
        censor_mode = 'end_time', end_time = 15, observed_p = None,
        num_features = 10, num_var = 2,
        treatment_group = False):
        """
        Factory class for producing simulated survival data.
        Current supports two forms of simulated data:
            Linear:
                Where risk is a linear combination of an observation's features
            Nonlinear (Gaussian):
                A gaussian combination of covariates

        Parameters:
            hr_ratio: lambda_max hazard ratio.
            average_death: average death time that is the mean of the
                Exponentional distribution.
            censor_mode: the method to calculate whether a patient is censored.
                Options: ['end_time', 'observed_p']
                'end_time': requires the parameter end_time, which is used to censor any patient with death_time > end_time
                'observed_p': requires the parammeter observed_p, which is the percentage of patients with observed death times
            end_time: censoring time that represents an 'end of study'. Any death
                time greater than end_time will be censored.
            num_features: size of observation vector. Default: 10.
            num_var: number of varaibles simulated data depends on. Default: 2.
            treatment_group: True or False. Include an additional covariate
                representing a binary treatment group.
        """

        self.hr_ratio = hr_ratio
        self.censor_mode = censor_mode
        self.end_time = end_time
        self.observed_p = observed_p
        self.average_death = average_death
        self.treatment_group = treatment_group
        self.m = int(num_features) + int(treatment_group)
        self.num_var = num_var

    def _linear_H(self,x):
        """
        Calculates a linear combination of x's features.
        Coefficients are 1, 2, ..., self.num_var, 0,..0]

        Parameters:
            x: (n,m) numpy array of observations

        Returns:
            risk: the calculated linear risk for a set of data x
        """
        # Make the coefficients [1,2,...,num_var,0,..0]
        b = np.zeros((self.m,))
        b[0:self.num_var] = range(1,self.num_var + 1)

        # Linear Combinations of Coefficients and Covariates
        risk = np.dot(x, b)
        return risk

    def _gaussian_H(self,x,
        c= 0.0, rad= 0.5):
        """
        Calculates the Gaussian function of a subset of x's features.

        Parameters:
            x: (n, m) numpy array of observations.
            c: offset of Gaussian function. Default: 0.0.
            r: Gaussian scale parameter. Default: 0.5.

        Returns:
            risk: the calculated Gaussian risk for a set of data x
        """
        max_hr, min_hr = log(self.hr_ratio), log(1.0 / self.hr_ratio)

        # Z = ( (x_0 - c)^2 + (x_1 - c)^2 + ... + (x_{num_var} - c)^2)
        z = np.square((x - c))
        z = np.sum(z[:,0:self.num_var], axis = -1)

        # Compute Gaussian
        risk = max_hr * (np.exp(-(z) / (2 * rad ** 2)))
        return risk

    def generate_data(self, N,
        method = 'gaussian', gaussian_config = {},
        **kwargs):
        """
        Generates a set of observations according to an exponentional Cox model.

        Parameters:
            N: the number of observations.
            method: the type of simulated data. 'linear' or 'gaussian'.
            guassian_config: dictionary of additional parameters for gaussian
                simulation.

        Returns:
            dataset: a dictionary object with the following keys:
                'x' : (N,m) numpy array of observations.
                't' : (N) numpy array of observed time events.
                'e' : (N) numpy array of observed time intervals.
                'hr': (N) numpy array of observed true risk.

        See:
        Peter C Austin. Generating survival times to simulate cox proportional
        hazards models with time-varying covariates. Statistics in medicine,
        31(29):3946-3958, 2012.
        """

        # Patient Baseline information
        data = np.random.uniform(low= -1, high= 1,
            size = (N,self.m))

        if self.treatment_group:
            data[:,-1] = np.squeeze(np.random.randint(0,2,(N,1)))
            print(data[:,-1])

        # Each patient has a uniform death probability
        p_death = self.average_death * np.ones((N,1))

        # Patients Hazard Model
        # \lambda(t|X) = \lambda_0(t) exp(H(x))
        #
        # risk = True log hazard ratio
        # log(\lambda(t|X) / \lambda_0(t)) = H(x)
        if method == 'linear':
            risk = self._linear_H(data)

        elif method == 'gaussian':
            risk = self._gaussian_H(data,**gaussian_config)

        # Center the hazard ratio so population dies at the same rate
        # independent of control group (makes the problem easier)
        risk = risk - np.mean(risk)

        # Generate time of death for each patient
        # currently exponential random variable
        death_time = np.zeros((N,1))
        for i in range(N):
            if self.treatment_group and data[i,-1] == 0:
                death_time[i] = np.random.exponential(p_death[i])
            else:
                death_time[i] = np.random.exponential(p_death[i]) / exp(risk[i])

        # If Censor_mode is 'observed_p': then find the end time in which observed_p percent of patients have an observed death
        if self.censor_mode is 'observed_p':
            if self.observed_p is None:
                raise ValueError("Parameter observed_p must be porivded if censor_mode is configured to 'observed_p'")
            end_time_idx = int(N * self.observed_p)
            self.end_time = np.sort(death_time.flatten())[end_time_idx]

        # Censor anything that is past end time
        censoring = np.ones((N,1))
        death_time[death_time > self.end_time] = self.end_time
        censoring[death_time == self.end_time] = 0

        # Flatten Arrays to Vectors
        death_time = np.squeeze(death_time)
        censoring = np.squeeze(censoring)

        dataset = {
            'x' : data.astype(np.float32),
            'e' : censoring.astype(np.int32),
            't' : death_time.astype(np.float32),
            'hr' : risk.astype(np.float32)
        }

        return dataset


In [0]:
!pip install collections

ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections


In [0]:
!pip install lasagne

In [0]:
!pip install

In [0]:
import h5py
import scipy.stats as st
from collections import defaultdict
import numpy as np
import pandas as pd
import copy

import lasagne

def load_datasets(dataset_file):
    datasets = defaultdict(dict)

    with h5py.File(dataset_file, 'r') as fp:
        for ds in fp:
            for array in fp[ds]:
                datasets[ds][array] = fp[ds][array][:]

    return datasets

def format_dataset_to_df(dataset, duration_col, event_col, trt_idx = None):
    xdf = pd.DataFrame(dataset['x'])
    if trt_idx is not None:
        xdf = xdf.rename(columns={trt_idx : 'treat'})

    dt = pd.DataFrame(dataset['t'], columns=[duration_col])
    censor = pd.DataFrame(dataset['e'], columns=[event_col])
    cdf = pd.concat([xdf, dt, censor], axis=1)
    return cdf

def standardize_dataset(dataset, offset, scale):
    norm_ds = copy.deepcopy(dataset)
    norm_ds['x'] = (norm_ds['x'] - offset) / scale
    return norm_ds

def bootstrap_metric(metric_fxn, dataset, N=100):
    def sample_dataset(dataset, sample_idx):
        sampled_dataset = {}
        for (key,value) in dataset.items():
            sampled_dataset[key] = value[sample_idx]
        return sampled_dataset

    metrics = []
    size = len(dataset['x'])

    for _ in range(N):
        resample_idx = np.random.choice(size, size=size, replace = True)
    
        metric = metric_fxn(**sample_dataset(dataset, resample_idx))
        metrics.append(metric)
    
    # Find mean and 95% confidence interval
    mean = np.mean(metrics)
    conf_interval = st.t.interval(0.95, len(metrics)-1, loc=mean, scale=st.sem(metrics))
    return {
        'mean': mean,
        'confidence_interval': conf_interval
    }

def get_optimizer_from_str(update_fn):
    if update_fn == 'sgd':
        return lasagne.updates.sgd
    elif update_fn == 'adam':
        return lasagne.updates.adam
    elif update_fn == 'rmsprop':
        return lasagne.updates.rmsprop

    return None

def calculate_recs_and_antirecs(rec_trt, true_trt, dataset, print_metrics=True):
    if isinstance(true_trt, int):
        true_trt = dataset['x'][:,true_trt]

    # trt_values = zip([0,1],np.sort(np.unique(true_trt)))
    trt_values = enumerate(np.sort(np.unique(true_trt)))
    equal_trt = [np.logical_and(rec_trt == rec_value, true_trt == true_value) for (rec_value, true_value) in trt_values]
    rec_idx = np.logical_or(*equal_trt)
    # original Logic
    # rec_idx = np.logical_or(np.logical_and(rec_trt == 1,true_trt == 1),
    #               np.logical_and(rec_trt == 0,true_trt == 0))

    rec_t = dataset['t'][rec_idx]
    antirec_t = dataset['t'][~rec_idx]
    rec_e = dataset['e'][rec_idx]
    antirec_e = dataset['e'][~rec_idx]

    if print_metrics:
        print("Printing treatment recommendation metrics")
        metrics = {
            'rec_median' : np.median(rec_t),
            'antirec_median' : np.median(antirec_t)
        }
        print("Recommendation metrics:", metrics)

    return {
        'rec_t' : rec_t, 
        'rec_e' : rec_e, 
        'antirec_t' : antirec_t, 
        'antirec_e' : antirec_e
    }


ImportError: ignored

In [0]:
!pip install theano

In [0]:
!pip install tensor

     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 3.1MB 6.0MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 296kB 39.4MB/s 
     |████████████████████████████████| 235kB 46.3MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 13.7MB 311kB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
  Created wheel for tensor: filename=tensor-0.3.6-cp36-none-any.whl size=71570 sha256=063d04cb50a2c6a4cae61caa5b63c35c684ab989371c4c19bb7f7855ee8e5539
  Stored in directory: /root/.cache/pip/wheels/32/ba/ac/dbd210e9db9e5d549b3d98c57713684c5768298f259b1925ef
  Created wheel for construct: filename=construct-2.10.56-cp36-none-any.whl size=56878 sha256=dbb24d818fc719abc7b489512fb7b8e69331ff27e1d9d1fe982a407107daecb9
  Stored in directory: /root/.cache/pip/wheels/28/7b/bb/5af0216178450142d64876ad635c9e64cb

In [37]:
!pip install deep_surv

ERROR: Could not find a version that satisfies the requirement deep_surv (from versions: none)
ERROR: No matching distribution found for deep_surv


In [0]:
from deep_surv import DeepSurv
from viz import plot_log
#from . import datasets